<a href="https://colab.research.google.com/github/nsriniva/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [3]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [4]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [5]:
print(top10)
display(df.head())
display(df.isna().sum())
display(df.shape)
df.SALE_PRICE.value_counts()


Index(['FLUSHING-NORTH', 'UPPER EAST SIDE (59-79)', 'UPPER EAST SIDE (79-96)',
       'BEDFORD STUYVESANT', 'BOROUGH PARK', 'UPPER WEST SIDE (59-79)',
       'GRAMERCY', 'ASTORIA', 'FOREST HILLS', 'UPPER WEST SIDE (79-96)'],
      dtype='object')


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING_CLASS_CATEGORY               0
TAX_CLASS_AT_PRESENT                  1
BLOCK                                 0
LOT                                   0
EASE-MENT                         23040
BUILDING_CLASS_AT_PRESENT             1
ADDRESS                               0
APARTMENT_NUMBER                  17839
ZIP_CODE                              1
RESIDENTIAL_UNITS                     1
COMMERCIAL_UNITS                      1
TOTAL_UNITS                           1
LAND_SQUARE_FEET                     53
GROSS_SQUARE_FEET                     1
YEAR_BUILT                           35
TAX_CLASS_AT_TIME_OF_SALE             0
BUILDING_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                            0
SALE_DATE                             0
dtype: int64

(23040, 21)

0          6909
10          199
800000      125
750000      121
650000      120
           ... 
5236177       1
229000        1
397218        1
4112000       1
1751425       1
Name: SALE_PRICE, Length: 3831, dtype: int64

In [6]:
#Drop EASE-MENT column as it's all NaNs

df = df.drop(columns=['EASE-MENT'])

df.shape

(23040, 20)

In [7]:
# Extract subset where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' 
# and the sale price was more than 100 thousand and less than 2 million.
df_family = df[(df.BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS') & ((df.SALE_PRICE > 100000) & (df.SALE_PRICE < 2000000) )]

In [8]:
# Drop the BUILDING_CLASS_CATEGORY column as it's now constant
# valued.
df_family = df_family.drop(columns=['BUILDING_CLASS_CATEGORY'])

In [9]:
# Examine the non numerical columns
df_family.describe(exclude='number')

,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,LAND_SQUARE_FEET,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,3151,3151,3151,3151,3151,1,3151,3151,3151
unique,5,6,2,13,3135,1,1035,11,91
top,4,OTHER,1,A1,117-05 LINCOLN STREET,RP.,"4,000",A1,01/31/2019
freq,1580,2990,3111,1185,2,1,289,1186,78


In [10]:
# LAND_SQUARE_FEET should be numeric, convert using the apply() method 
df_family.LAND_SQUARE_FEET = df_family.LAND_SQUARE_FEET.apply(lambda x: int(x.replace(',','')))

df_family.LAND_SQUARE_FEET.dtype

dtype('int64')

In [11]:
display(df_family.isna().sum())
display(df_family.APARTMENT_NUMBER.value_counts())

BOROUGH                              0
NEIGHBORHOOD                         0
TAX_CLASS_AT_PRESENT                 0
BLOCK                                0
LOT                                  0
BUILDING_CLASS_AT_PRESENT            0
ADDRESS                              0
APARTMENT_NUMBER                  3150
ZIP_CODE                             0
RESIDENTIAL_UNITS                    0
COMMERCIAL_UNITS                     0
TOTAL_UNITS                          0
LAND_SQUARE_FEET                     0
GROSS_SQUARE_FEET                    0
YEAR_BUILT                           0
TAX_CLASS_AT_TIME_OF_SALE            0
BUILDING_CLASS_AT_TIME_OF_SALE       0
SALE_PRICE                           0
SALE_DATE                            0
dtype: int64

RP.    1
Name: APARTMENT_NUMBER, dtype: int64

In [12]:
# The APARTMENT_NUMBER column has 3150 NaN entries and only one
# non NaN value - drop the APARTMENT_NUMBER column
# Also drop high cardinality address column
df_family = df_family.drop(columns=['APARTMENT_NUMBER','ADDRESS'])

In [13]:
#Convert SALE_DATE column to datetime
df_family.SALE_DATE = df_family.SALE_DATE.astype('datetime64[ns]')


In [14]:
# Do train/test split. 

# Use data from January — March 2019 to train. 
df_family_train = df_family[(df_family.SALE_DATE >= 'JANUARY 2019') & (df_family.SALE_DATE < 'APRIL 2019')] 

# Use data from April 2019 to test.
df_family_test = df_family[df_family.SALE_DATE >= 'APRIL 2019'] 

In [15]:
display(df_family.shape, df_family_train.shape, df_family_test.shape)

(3151, 17)

(2507, 17)

(644, 17)

In [16]:
#Examine the non numeric columns
df_family_train.describe(exclude='number')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  


,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,2507,2507,2507,2507,2507,2507
unique,5,6,2,13,11,68
top,4,OTHER,1,A1,A1,2019-01-31 00:00:00
freq,1204,2382,2476,919,919,78
first,NaN,NaN,NaN,NaN,NaN,2019-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,2019-03-30 00:00:00


In [17]:
# Drop SALE_DATE(high cardinality column) from train and test dataset
drop_columns = lambda train, test, cols: (train.drop(columns=cols), test.drop(columns=cols))


df_family_train, df_family_test = drop_columns(df_family_train, df_family_test,['SALE_DATE'])

In [18]:
display(df_family_train.describe(exclude='number'))
display(df_family_train.shape, df_family_test.shape)



,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,BUILDING_CLASS_AT_TIME_OF_SALE
count,2507,2507,2507,2507,2507
unique,5,6,2,13,11
top,4,OTHER,1,A1,A1
freq,1204,2382,2476,919,919


(2507, 16)

(644, 16)

In [19]:
import category_encoders as ce

# Perform One Hot Encoding of categorical features
encoder = ce.OneHotEncoder(use_cat_names=True)
df_family_train = encoder.fit_transform(df_family_train)
df_family_test = encoder.transform(df_family_test)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [20]:
display(df_family_train.shape, df_family_test.shape)


(2507, 48)

(644, 48)

In [21]:
from sklearn.feature_selection import SelectKBest, f_regression

get_xy = lambda df,target='SALE_PRICE': (df.drop(columns=[target]), df[target])

# Get X_train, X_test matrices and y_train, y_test vectors
X_train, y_train = get_xy(df_family_train)
X_test, y_test = get_xy(df_family_test)



In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

# For parameter k, use SelectKBest to compute the k best
# features and use those to train a LinearRegression
# model.
# Use the trained model with X_test to compute y_pred and compare
# against y_test to compute and return MAE along with the list
# of k chosen features
def select_and_fit(X_train, y_train, X_test, y_test, k):
    
  selector = SelectKBest(score_func=f_regression, k=k)
  X_train_selected = selector.fit_transform(X_train, y_train)
  X_test_selected = selector.transform(X_test)

  model = LinearRegression()
  model.fit(X_train_selected, y_train)
  y_pred = model.predict(X_test_selected)
  return mean_absolute_error(y_test, y_pred), list(X_train.columns[selector.get_support()])

best_features=[]
best_k = 0
best_mae = 1000000
# Loop through k from 1 to 47 and compare MAEs to determine the best
# k features(best_features) with the lowest MAE
for k in range(1, len(X_train.columns)+1):
    mae, feat = select_and_fit(X_train, y_train, X_test, y_test, k)
    if mae < best_mae:
      best_mae = mae
      best_k = k
      best_features = feat
    #print(f'Test Mean Absolute Error: ${mae:,.0f} \n')

print(f'best_k = {best_k}\nbest MAE = ${best_mae:,.0f}\nbest features = {feat}')

best_k = 29
best MAE = $153,863
best features = ['BOROUGH_3', 'BOROUGH_4', 'BOROUGH_2', 'BOROUGH_5', 'BOROUGH_1', 'NEIGHBORHOOD_OTHER', 'NEIGHBORHOOD_FLUSHING-NORTH', 'NEIGHBORHOOD_BEDFORD STUYVESANT', 'NEIGHBORHOOD_FOREST HILLS', 'NEIGHBORHOOD_BOROUGH PARK', 'NEIGHBORHOOD_ASTORIA', 'TAX_CLASS_AT_PRESENT_1', 'TAX_CLASS_AT_PRESENT_1D', 'BLOCK', 'LOT', 'BUILDING_CLASS_AT_PRESENT_A9', 'BUILDING_CLASS_AT_PRESENT_A1', 'BUILDING_CLASS_AT_PRESENT_A5', 'BUILDING_CLASS_AT_PRESENT_A0', 'BUILDING_CLASS_AT_PRESENT_A2', 'BUILDING_CLASS_AT_PRESENT_A3', 'BUILDING_CLASS_AT_PRESENT_S1', 'BUILDING_CLASS_AT_PRESENT_A4', 'BUILDING_CLASS_AT_PRESENT_A6', 'BUILDING_CLASS_AT_PRESENT_A8', 'BUILDING_CLASS_AT_PRESENT_B2', 'BUILDING_CLASS_AT_PRESENT_S0', 'BUILDING_CLASS_AT_PRESENT_B3', 'ZIP_CODE', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE_A9', 'BUILDING_CLASS_AT_TIME_OF_SALE_A1', 'BUIL

In [23]:
alphas = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]

In [24]:
# Use RidgeCV to determine the best alpha and fit
# with the identified best_features in the
# X_train matrix.
from sklearn.linear_model import RidgeCV
ridge = RidgeCV(alphas=alphas, normalize=True)
ridge.fit(X_train[best_features], y_train)
ridge.alpha_

0.001

In [25]:
# Use the trained ridge model to predict against
# the best_features from the X_test matrix
y_pred = ridge.predict(X_test[best_features])

In [26]:
# Compute and print the test MAE
print(f'Test MAE = ${mean_absolute_error(y_test, y_pred):,.0f}')

Test MAE = $153,950
